In [1]:
import rpy2.rinterface
from rpy2.robjects import r, pandas2ri

In [2]:
import sys as sys
sys.path.append('/Users/grlurton/Documents/bluesquare/data_pipelines/src')

import audit.dhis as dhis 
import audit.completeness as cplt
import pandas as pd

In [3]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2
pandas2ri.activate()

In [4]:
hivdr = dhis.dhis_instance('hivdr', 'grlurton', 'localhost')

In [5]:
%%R
library(ggplot2)
library(zoo)
library(dplyr)
library(data.table)

source("/Users/grlurton/Documents/hivdr/patient_data.R")
data_dir <- "/Users/grlurton/data/dhis/rdc/hivdr/"
metadata_dir <- "/Users/grlurton/data/dhis/rdc/hivdr/metadata/"

load_metadata(metadata_dir)

cordaid <- readRDS(paste0(data_dir, 'Cordaid_TRAITEMENTS.rds'))
cordaid <- merge(cordaid, M_org_units, by.x = 'orgUnit', by.y='id', all.y = FALSE)
pnls <- readRDS(paste0(data_dir, 'PNLS.rds'))
pnls <- merge(pnls, M_org_units, by.x = 'orgUnit', by.y='id', all.y = FALSE)

pnls_id <- 'Dd2G5zI0o0a'
cordaid_id <- 'Yj8caUQs178'

ancient_cat_option <-  c('vZ6Os4BJvum','ggod3chlUCG')
ancient_cat_combo <- M_category_combos$CatComboOpt_id[M_category_combos$CatOpt_id.1 %in% ancient_cat_option]

/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attachement du package : ‘zoo’


  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attachement du package : ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rint

## CSV1 - Total Patients



| level_2_name  | level_2_id    | level_3_name  | level_3_id    | level_3_id   | source        |  periods      | value |
| ------------- | ------------- | ------------- | ------------- |------------- | ------------- | ------------- | ----- |


### Build the total number of patients for Cordaid

In [6]:
def get_hivdrable(de_id):
    data = hivdr.get_data(de_id)
    data = data[['dataelementid', 'monthly', 'uidorgunit', 'catcomboid', 'value']]
    data.columns = ['dataElement' , 'period' , 'orgUnit' , 'categoryOptionCombo', 'value']
    return data
    
cordaid_exp = get_hivdrable('Yj8caUQs178')
%Rpush cordaid_exp

In [7]:
%%R
cordaid_exp$value <- as.numeric(cordaid_exp$value)
cordaid_full <- cordaid_exp %>% group_by(period, orgUnit) %>% summarize('value' = sum(value))
cordaid_full <- make_full_unique_set(cordaid_full)
adhoc_period <- data.frame('month' = unique(cordaid_full$periods),
                           'periods' = seq(201701,201712))
cordaid_full$source <- 'cordaid'
cordaid_imputed <- cordaid_full %>% group_by(orgUnit) %>% do(predict_missing(.))

|===================================================== |100% ~0 s remaining     

### Build the total number of patients for PNLS

In [8]:
pnls_exp = get_hivdrable('Dd2G5zI0o0a')
%Rpush pnls_exp

In [9]:
%%R
pnls_exp$value <- as.numeric(pnls_exp$value)
pnls_full <- pnls_exp %>% group_by(period, orgUnit) %>% summarize('value' = sum(value))
pnls_full <- make_full_unique_set(pnls_full)
adhoc_period <- data.frame('month' = unique(pnls_full$periods), 'periods' = c(seq(201701,201712),seq(201801,201812))[1:length(unique(pnls_full$periods))])
pnls_full$source <- 'pnls'
pnls_imputed <- pnls_full %>% group_by(orgUnit) %>% do(predict_missing(.))

|======================================================|100% ~0 s remaining     

### Build the total number of patients combined with both sources

In [10]:
%%R
## Making consolidation and imputation on set with both PNLS and CORDAID data
common_set <- build_common_set(cordaid_imputed, pnls_imputed, 'cordaid', 'pnls')
common_set$month <- period_monthly(common_set$periods)
common_set <- common_set[!is.na(common_set$month), ]
completed_data_cordaid_pnls_total <- completed_data(common_set, 'cordaid', 'pnls')
completed_data_cordaid_pnls_total$value[completed_data_cordaid_pnls_total$value > 1e+05] <- NA
adhoc_period <- data.frame('month' = unique(completed_data_cordaid_pnls_total$periods), 
                           'periods' = c(seq(201701,201712),seq(201801,201812))[1:length(unique(completed_data_cordaid_pnls_total$periods))])
imputed_data_cordaid_pnls_total <- completed_data_cordaid_pnls_total %>% group_by(orgUnit) %>% do(predict_missing(.))
imputed_data_cordaid_pnls_total <- imputed_data_cordaid_pnls_total[,colnames(pnls_imputed)]

|===================================================== | 99% ~0 s remaining     

In [11]:
%%R
## Keeping only data from CORDAID and PNLS with no value in other sources, and combining

imputed_data_cordaid_pnls_total$value_id <- paste0(imputed_data_cordaid_pnls_total$period, imputed_data_cordaid_pnls_total$orgUnit)
pnls_imputed$value_id <- paste0(pnls_imputed$period, pnls_imputed$orgUnit)
cordaid_imputed$value_id <- paste0(cordaid_imputed$period, cordaid_imputed$orgUnit)


pnls_unique <- subset(pnls_imputed, !(value_id %in% imputed_data_cordaid_pnls_total$value_id))
pnls_unique$source <- 'combine'
cordaid_unique <- subset(cordaid_imputed, !(value_id %in% imputed_data_cordaid_pnls_total$value_id))
cordaid_unique$source <- 'combine'

pnls_imputed$source <- 'pnls'
cordaid_imputed$source <- 'cordaid'

imputed_data_cordaid_pnls_total$source <- 'combine'

col_keep <- c('periods', 'orgUnit', 'value', 'source')
final_imputation <- rbind(data.frame(imputed_data_cordaid_pnls_total[,col_keep]),
                          data.frame(pnls_unique[,col_keep]),
                          data.frame(cordaid_unique[,col_keep]),
                          data.frame(pnls_imputed[,col_keep]),
                          data.frame(cordaid_imputed[,col_keep])
                         )
final_imputation_hier <- merge(final_imputation, M_hierarchy, by.x = 'orgUnit', by.y = 'id')

### Output DS

In [12]:
%%R
ds1 <- final_imputation_hier %>% group_by(level_2_name, level_2_id, level_3_name, level_3_id, source, periods) %>% do(data.frame(value = sum(.$value)))
write.csv(ds1, '../data/hivdr_patients.csv')

|===================================================== |100% ~0 s remaining     

In [170]:
%%R
N_fac <- final_imputation_hier %>% group_by(level_2_name, level_2_id, level_3_name, level_3_id) %>% do(data.frame(value = length(unique(.$orgUnit))))
write.csv(N_fac, '../data/hivdr_facilities.csv')

In [171]:
%%R
sum(N_fac$value)

[1] 2937


## Treatment Lines Distribution

| level_2_name  | level_2_id    | level_3_name  | level_3_id    | level_3_id   |   periods      | value |
| ------------- | ------------- | ------------- | ------------- |------------- |  ------------- | ----- |


We look at this with two sourcses:
* PNLS - number of patients reported for each line
* CORDAID

We end up keeping the CORDAID data as it is the one with best completeness

In [14]:
%%R
art_pnls <- c('PNLS-DRUG-ABC + 3TC + EFV','PNLS-DRUG-ABC + 3TC + EFV sex','PNLS-DRUG-ABC + 3TC + LPV/r','PNLS-DRUG-ABC + 3TC + LPV/r sex','PNLS-DRUG-ABC + 3TC + NVP',
              'PNLS-DRUG-ABC + 3TC + NVP sex', 'PNLS-DRUG-AZT + 3TC + LPV/r', 'PNLS-DRUG-AZT + 3TC + LPV/r sex','PNLS-DRUG-AZT+3TC+EFV', 'PNLS-DRUG-AZT+3TC+EFV sex',
              'PNLS-DRUG-AZT+3TC+NVP', 'PNLS-DRUG-AZT+3TC+NVP sex','PNLS-DRUG-TDF + 3TC + LPV/r', 'PNLS-DRUG-TDF + 3TC + LPV/r sex', 'PNLS-DRUG-TDF + FTC + NVP', 
              'PNLS-DRUG-TDF + FTC + NVP sex','PNLS-DRUG-TDF+ FTC + EFV', 'PNLS-DRUG-TDF+ FTC + EFV sex', 'PNLS-DRUG-TDF+3TC+EFV sex', 'PNLS-DRUG-TDF+3TC+NVP sex','PNLS-DRUG-Autres (à préciser)',
              'PNLS-DRUG-Autres (à préciser) sex')

art_cordaid <- c('ABC + 3TC + EFV', 'ABC + 3TC + LPV/r', 'ABC + 3TC + NVP', 'Autres (à préciser)', 'AZT + 3TC + LPV/r', 'AZT+3TC+ EFV',
                 'AZT+3TC+NVP', 'TDF + 3TC + LPV/r', 'TDF + FTC + NVP', 'TDF+ FTC + EFV', 'TDF+3TC+EFV', 'TDF+3TC+NVP', 'TDF+FTC+LPV+rt')

In [15]:
%%R 
lines_dic <- data.frame()
no_match <- c()
for(line in art_cordaid){
    line2 <- gsub('\\+', '\\\\+', line)
    line2 <- gsub('\\à', '\\\\à', line2)
    line2 <- gsub('\\é', '\\\\é', line2)
    line2 <- gsub('\\(', '\\\\(', line2)
    line2 <- gsub('\\)', '\\\\)', line2)
    match <- grep(line2, art_pnls, value=TRUE)
    if (length(match) >0){
        addin <- data.frame('pnls' = match, 
                            'cordaid' = line)
        lines_dic <- rbind(lines_dic, addin)
    }
    if(length(match) == 0){
        no_match <- c(no_match, line)
    }
}

addin <- data.frame('pnls' = c('PNLS-DRUG-AZT+3TC+EFV', 'PNLS-DRUG-AZT+3TC+EFV sex'), 'cordaid' = "AZT+3TC+ EFV")
lines_dic <- rbind(lines_dic, addin)

addin <- data.frame('pnls' = c(''), 'cordaid' =  "TDF+FTC+LPV+rt")
lines_dic <- rbind(lines_dic, addin)

lines_dic <- lines_dic[grep('sex', lines_dic$pnls),]

In [16]:
%%R 
lines <- data.frame()

In [17]:
%%R 
dat_common <- unique(common_set$orgUnit)

In [18]:
%R source("/Users/grlurton/Documents/hivdr/patient_data.R")

lines_dic = r.lines_dic
for line in lines_dic.cordaid :
    print(line)
    cordaid_id = hivdr.dataelement.uid[hivdr.dataelement.name == line].values[0]
    pnls_line = lines_dic.pnls[lines_dic.cordaid == line].values[0]
    pnls_id = hivdr.dataelement.uid[hivdr.dataelement.name == pnls_line].values[0]
    data_cordaid = get_hivdrable(cordaid_id)
    data_cordaid.value = pd.to_numeric(data_cordaid.value)
    data_cordaid = data_cordaid.groupby(['dataElement','period','orgUnit'])['value'].sum().reset_index()
    data_pnls = get_hivdrable(pnls_id)
    data_pnls.value = pd.to_numeric(data_pnls.value)
    data_pnls = data_pnls.groupby(['dataElement','period','orgUnit'])['value'].sum().reset_index()
    data_cordaid['source'] = 'cordaid'
    data_pnls['source'] = 'pnls'
    data_common = data_pnls[data_pnls.orgUnit.isin(r.dat_common)].append(data_cordaid[data_cordaid.orgUnit.isin(r.dat_common)])
    data_pnls_unique = data_pnls[~data_pnls.orgUnit.isin(r.dat_common)][['period', 'orgUnit', 'value','source']]
    data_cordaid_unique = data_cordaid[~data_cordaid.orgUnit.isin(r.dat_common)][['period', 'orgUnit', 'value','source']]
    %Rpush data_common
    %R data_common$month <- period_monthly(data_common$period)
    %R completed_data_cordaid_pnls_total <- completed_data(data_common, 'cordaid', 'pnls')
    %R adhoc_period <- data.frame('month' = unique(completed_data_cordaid_pnls_total$periods), 'periods' = c(seq(201701,201712),seq(201801,201812))[1:length(unique(completed_data_cordaid_pnls_total$periods))])
    %R imputed_data_cordaid_pnls_total <- completed_data_cordaid_pnls_total %>% group_by(orgUnit) %>% do(predict_missing(.))
    %Rpush line
    %R imputed_data_cordaid_pnls_total['line'] <- line
    imputed_data_cordaid_pnls_total = r.imputed_data_cordaid_pnls_total
    %R imputed_data_cordaid_pnls_total = imputed_data_cordaid_pnls_total[,c('periods', 'orgUnit', 'value', 'name', 'level_2_id', 'level_2_name', 'level_3_id', 'level_3_name', 'line')]
    %R imputed_data_cordaid_pnls_total$source = 'combine'
    data_pnls_unique = data_pnls_unique.merge(hivdr.orgunitstructure, left_on = 'orgUnit', right_on = 'organisationunituid')
    cols = ['period', 'orgUnit', 'value', 'namelevel3', 'uidlevel2', 'namelevel2', 'uidlevel3','source', 'namelevel5']
    data_pnls_unique = data_pnls_unique[cols]
    data_cordaid_unique = data_cordaid_unique.merge(hivdr.orgunitstructure, left_on = 'orgUnit', right_on = 'organisationunituid')
    data_cordaid_unique =data_cordaid_unique[cols]
    data_pnls_unique['line'] = line
    data_cordaid_unique['line'] = line
    %Rpush data_pnls_unique data_cordaid_unique
    %R data_cordaid_unique <- data.frame(data_cordaid_unique)
    %R data_pnls_unique <- data.frame(data_pnls_unique)
    %R colnames(data_pnls_unique) <- colnames(data_cordaid_unique) <- c('periods', 'orgUnit', 'value', 'level_3_name', 'level_2_id', 'level_2_name', 'level_3_id', 'source', 'name','line')
    %R out = rbind(data.frame(imputed_data_cordaid_pnls_total), data.frame(data_cordaid_unique), data.frame(data_pnls_unique))
    %R lines <-rbind(lines, out)

ABC + 3TC + EFV


|===================================================== |100% ~0 s remaining     

|====================================================  | 97% ~0 s remaining     

ABC + 3TC + LPV/r


|======================================================|100% ~0 s remaining     

...

|===================================================== | 99% ~0 s remaining     

TDF+3TC+NVP


|===================================================== | 99% ~0 s remaining     

AZT+3TC+ EFV


|======================================================|100% ~0 s remaining     

In [19]:
%%R
lines <- lines %>% group_by(level_2_id, level_2_name, level_3_id, level_3_name, periods, line) %>% do(data.frame(value = sum(.$value)))
write.csv(lines, '../data/hivdr_lines.csv')

|===================================================== |100% ~0 s remaining     

## Looking at stock outs

In [20]:
%%R 
exclude <- c('PNLS-DRUG-CTX 480 / 960 mg ces - Bt 500 ces', 'PNLS-DRUG-CTX 480 mg ces - Bt 1000 ces',
             'PNLS-DRUG-Hepatitis, HBsAg, Determine Kit, 100 Tests','PNLS-DRUG-Hepatitis, HCV, Rapid Device, Serum/Plasma/Whole Blood, kit de 40 Tests',
             'PNLS-DRUG-HIV 1/2, Double Check Gold, Kit de 100 test','PNLS-DRUG-HIV 1+2, Determine Complete, Kit de 100 tests',
             'PNLS-DRUG-HIV 1+2, Uni-Gold HIV, Kit de 20 tests', 'PNLS-DRUG-INH 100 mg; 300 mg - Cés', 'PNLS-DRUG-INH 50mg/5 ml - Sol.Orale',
             'PNLS-DRUG-Syphilis RPR Kit, kit de 100 Tests Determine syph','PNLS-DRUG-CTX 96 mg / ml - Inj')

In [21]:
%%R
drugs_pnls <- M_data_sets[M_data_sets$categoryCombo.id == 'Q3ONIkE9JN5' & !(M_data_sets$DE_name %in% exclude),]
pnls_drugs <- pnls[pnls$dataElement %in% drugs_pnls$DE_id, ]

pnls_drugs_cc <- merge(pnls_drugs, M_category_combos[,c('CatComboOpt_id', 'CatOpt_name.1')], by.x ='categoryOptionCombo', by.y = 'CatComboOpt_id'  )
pnls_drugs <- merge(pnls_drugs_cc, M_data_sets[,c('DE_name', 'DE_id')], by.x = 'dataElement', by.y = 'DE_id')

In [22]:
cc_rs = hivdr.categoryoptioncombo.categoryoptioncomboid[hivdr.categoryoptioncombo.name.str.contains('RS')]
cc = hivdr.categorycombos_optioncombos.categorycomboid[hivdr.categorycombos_optioncombos.categoryoptioncomboid == cc_rs.iloc[0]]
rs_pnls = hivdr.dataelement[hivdr.dataelement.categorycomboid == cc.iloc[0]]

In [23]:
rs_cordaid = hivdr.dataelement[hivdr.dataelement.name.str.contains('RS')]

In [24]:
rs_cordaid['stand_name']=rs_cordaid.name.str.replace("RS -",'')
rs_cordaid['stand_name']=rs_cordaid['stand_name'].str.replace(' ','').str.lower()
rs_pnls['stand_name']=rs_pnls.name.str.replace(" ",'')
rs_pnls.stand_name = rs_pnls.stand_name.str.replace("PNLS-DRUG-",'').str.lower()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [25]:
log_dic = rs_pnls.merge(rs_cordaid, on='stand_name', suffixes=['_pnls', '_cordaid'])

In [26]:
rs_pnls.columns = ['uid_pnls', 'name_pnls', 'dataelementid_pnls' , 'categorycomboid_pnls', 'stand_name']
rs_cordaid.columns = ['uid_cordaid', 'name_cordaid', 'dataelementid_cordaid' , 'categorycomboid_cordaid', 'stand_name']

log_dic = log_dic.append(rs_pnls.drop('stand_name', axis=1).loc[404].append(rs_cordaid.loc[1079]).to_frame().transpose())
log_dic = log_dic.append(rs_pnls.drop('stand_name', axis=1).loc[418].append(rs_cordaid.loc[1089]).to_frame().transpose())

log_dic = log_dic[~log_dic.name_pnls.isin(r.exclude)]

In [27]:
rs_pnls_cc = hivdr.categoryoptioncombo[hivdr.categoryoptioncombo.name.str.contains('RS')].uid.iloc[0]

In [28]:
data = pd.DataFrame()
for line in log_dic.name_cordaid:
    id_cordaid = log_dic.uid_cordaid[log_dic.name_cordaid == line].iloc[0]
    id_pnls = log_dic.uid_pnls[log_dic.name_cordaid == line].iloc[0]
    cordaid_data = get_hivdrable(id_cordaid)
    pnls_data = get_hivdrable(id_pnls)
    pnls_data = pnls_data[pnls_data.categoryOptionCombo == rs_pnls_cc]
    data = data.append(cordaid_data).append(pnls_data)

In [29]:
def any_rs(data):
    return sum(pd.to_numeric(data.value)) > 0

fac_rupture = data.groupby(['period','orgUnit']).apply(any_rs).reset_index()
fac_rupture.columns = ['period', 'orgUnit', 'fac_rupture']

In [30]:
fac_rupture = fac_rupture.merge(hivdr.orgunitstructure, left_on = 'orgUnit', right_on = 'organisationunituid')
perc_zone_rs = fac_rupture.groupby(['period','uidlevel2','uidlevel3','namelevel2','namelevel3'])['fac_rupture'].sum().reset_index()

In [31]:
perc_zone_rs.to_csv('../data/hivdr_facilities_stockout.csv.csv')

### Sorties

In [32]:
cc_rs = hivdr.categoryoptioncombo.categoryoptioncomboid[hivdr.categoryoptioncombo.name.str.contains('Sortie')]
cc = hivdr.categorycombos_optioncombos.categorycomboid[hivdr.categorycombos_optioncombos.categoryoptioncomboid == cc_rs.iloc[0]]
so_pnls = hivdr.dataelement[hivdr.dataelement.categorycomboid == cc.iloc[0]]

so_cordaid = hivdr.dataelement[hivdr.dataelement.name.str.contains('SO -')]

In [33]:
so_cordaid['stand_name']=so_cordaid.name.str.replace("SO -",'')
so_cordaid['stand_name']=so_cordaid['stand_name'].str.replace(' ','').str.lower()
so_pnls['stand_name']=so_pnls.name.str.replace(" ",'')
so_pnls.stand_name = so_pnls.stand_name.str.replace("PNLS-DRUG-",'').str.lower()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [34]:
log_dic = so_pnls.merge(so_cordaid, on='stand_name', suffixes=['_pnls', '_cordaid'])

In [35]:
so_pnls.columns = ['uid_pnls', 'name_pnls', 'dataelementid_pnls' , 'categorycomboid_pnls', 'stand_name']
so_cordaid.columns = ['uid_cordaid', 'name_cordaid', 'dataelementid_cordaid' , 'categorycomboid_cordaid', 'stand_name']

log_dic = log_dic.append(so_pnls.drop('stand_name', axis=1).loc[404].append(so_cordaid.loc[1177]).to_frame().transpose())
log_dic = log_dic.append(so_pnls.drop('stand_name', axis=1).loc[418].append(so_cordaid.loc[1223]).to_frame().transpose())
log_dic = log_dic.append(so_pnls.drop('stand_name', axis=1).loc[408].append(so_cordaid.loc[1217]).to_frame().transpose())

log_dic = log_dic[~log_dic.name_pnls.isin(r.exclude)]

In [37]:
so_pnls_cc = hivdr.categoryoptioncombo[hivdr.categoryoptioncombo.name.str.contains('Sortie')].uid.iloc[0]

In [178]:
data_conso = pd.DataFrame()
for line in log_dic.name_cordaid:
    id_cordaid = log_dic.uid_cordaid[log_dic.name_cordaid == line].iloc[0]
    id_pnls = log_dic.uid_pnls[log_dic.name_cordaid == line].iloc[0]
    cordaid_data = get_hivdrable(id_cordaid)
    pnls_data = get_hivdrable(id_pnls)
    pnls_data = pnls_data[pnls_data.categoryOptionCombo == so_pnls_cc]
    data_conso = data_conso.append(cordaid_data).append(pnls_data)

In [179]:
data_conso.value = pd.to_numeric(data_conso.value)
data_conso['pills_reported'] = data_conso.value % 30 == 0 

In [180]:
data_conso['boxes']=0
data_conso.boxes[data_conso.pills_reported == True] = data_conso.value[data_conso.pills_reported == True] / 30
data_conso.boxes[data_conso.pills_reported == False] = data_conso.value[data_conso.pills_reported == False]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [181]:
data_conso = data_conso.merge(hivdr.orgunitstructure, left_on = 'orgUnit', right_on = 'organisationunituid')
data_conso = data_conso.merge(hivdr.dataelement, left_on = 'dataElement' , right_on = 'uid')

In [182]:
data_conso['stand_name'] = data_conso.name.str.replace('SO - ', '')
data_conso.stand_name = data_conso.stand_name.str.replace("PNLS-DRUG-",'').str.lower()
data_conso.stand_name = data_conso.stand_name.str.replace("abc\/3tc \( 300\/150 mg\) - 60 ces",  "abc/3tc(300/150 mg) - 60 ces")
data_conso.stand_name = data_conso.stand_name.str.replace("lvp\/r \(133\/33 mg\) - 120 ces",  "lvp/r(133/33 mg) - 120 ces")
data_conso.stand_name = data_conso.stand_name.str.replace("é",  "e")
data_conso.stand_name = data_conso.stand_name.str.replace("azt\\/3tc 60\\/30 mg ces disp - 60ces",  "azt/3tc 60/30 mg ces disp - 60 ces")
data_conso.stand_name = data_conso.stand_name.str.replace("tdf\\/ftc \\(300\\/200 mg\\) - 30 ces",  "tdf/ftc(300/200 mg) -")
data_conso.stand_name = data_conso.stand_name.str.replace('tdf\\/3tc \\(300\\/300 mg\\) - 30 ces', 'tdf/3tc(300/300 mg) - 30 ces')
data_conso.stand_name = data_conso.stand_name.str.replace('tdf\\/3tc\\/efv \\(300\\/300\\/600 mg\\) - 30 ces', 'tdf/3tc/efv(300/300/600 mg) - 30 ces')

In [186]:
len(data_conso)

246560

In [184]:
data_conso.columns

Index(['dataElement', 'period', 'orgUnit', 'categoryOptionCombo', 'value',
       'pills_reported', 'boxes', 'organisationunituid', 'level', 'uidlevel1',
       'uidlevel2', 'uidlevel3', 'uidlevel4', 'uidlevel5', 'namelevel1',
       'namelevel2', 'namelevel3', 'namelevel4', 'namelevel5', 'uid', 'name',
       'dataelementid', 'categorycomboid', 'stand_name'],
      dtype='object')

In [185]:
data_conso = data_conso.drop_duplicates(['stand_name','organisationunituid','period'])

In [187]:
%matplotlib inline
out = data_conso.groupby(['stand_name', 'period','uidlevel2','uidlevel3','namelevel2','namelevel3' ]).boxes.sum().reset_index()

In [188]:
import numpy as np
out.boxes[out.boxes > 6000] = out.boxes[out.boxes > 5000] / 30

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [189]:
out.to_csv('../data/hivdr_conso.csv')

In [ ]:
## server


function(input, output) {
  
  ## SOURCE ######################################################################################################
  
  ## DATA 
  
  df1_region_cordaid_filt <- reactive({
    m <- df1_region_cordaid %>%
      filter(
        periods == input$period)
    m <- left_join(coordinates_region, m, by = c('id' = 'level_2_id'))
    m$value[is.na(m$value)] <- 0
    m
  })
  
  
  df1_region_pnls_filt <- reactive({
    m <- df1_region_pnls %>%
      filter(
        periods == input$period)
    m <- left_join(coordinates_region, m, by = c('id' = 'level_2_id'))
    m$value[is.na(m$value)] <- 0
    m
  })
  
  df1_region_combine_filt <- reactive({
    m <- df1_region_combine %>%
      filter(
        periods == input$period)
    m <- left_join(coordinates_region, m, by = c('id' = 'level_2_id'))
    m$value[is.na(m$value)] <- 0
    m
  })
  
  df1_all_filt <- reactive({
    m <- df1_all %>%
      filter(
        periods == input$period,
        level_2_name == input$region)
    m$pnls[is.na(m$pnls)] <- 0
    m$cordaid[is.na(m$cordaid)] <- 0
    m$combine[is.na(m$combine)] <- 0
    m
  })
  
  df1_region_all_filt <- reactive({
    m <- df1_region_all %>%
      filter(
        periods == input$period)
    m$pnls[is.na(m$pnls)] <- 0
    m$cordaid[is.na(m$cordaid)] <- 0
    m$combine[is.na(m$combine)] <- 0
    m
  })
  
  ## PLOT 
  
  output$plot1 <- renderPlot({
    
    ggplot(df1_region_pnls_filt())+
      geom_polygon(aes(x  = long, y = lat , 
                       group=id, 
                       fill = value))+
      theme_minimal() +
      coord_map() + 
      scale_fill_gradient(name = "Number of patients",
                          low = '#eeeeee', high = '#ef0404',
                          space = "Lab",
                          na.value = "grey50", guide = "colourbar")+
      ggtitle("PNLS")+
      theme(plot.title = element_text(size=20, hjust = 0.55),
            legend.position = "bottom",
            legend.key.width = unit(2, "cm"),
            legend.text=element_text(size=13),
            legend.title=element_text(size=15))
    
  })
  
  output$plot2 <- renderPlot({
    
    ggplot(df1_region_cordaid_filt())+
      geom_polygon(aes(x  = long, y = lat , 
                       group=id, 
                       fill = value))+
      theme_minimal() +
      coord_map() + 
      scale_fill_gradient(name = "Number of patients",
                          low = '#eeeeee', high = '#ef0404',
                          space = "Lab",
                          na.value = "grey50", guide = "colourbar") +
      ggtitle("CORDAID")+
      theme(plot.title = element_text(size=20, hjust = 0.55),
            legend.position = "bottom",
            legend.key.width = unit(2, "cm"),
            legend.text=element_text(size=13),
            legend.title=element_text(size=15))
    
  })
  
  output$plot3 <- renderPlot({
    
    ggplot(df1_region_combine_filt())+
      geom_polygon(aes(x  = long, y = lat , 
                       group=id, 
                       fill = value))+
      theme_minimal() +
      coord_map() + 
      scale_fill_gradient(name = "Number of patients",
                          low = '#eeeeee', high = '#ef0404',
                          space = "Lab",
                          na.value = "grey50", guide = "colourbar")+
      ggtitle("COMBINED")+
      theme(plot.title = element_text(size=20, hjust = 0.55),
            legend.position = "bottom",
            legend.key.width = unit(2, "cm"),
            legend.text=element_text(size=13),
            legend.title=element_text(size=15))
    
  })
  
  output$datatable1 <- DT::renderDataTable({
    dttbl <- df1_region_all_filt()
    
    DT::datatable(
      dttbl, filter="top", selection="multiple", escape=FALSE,
      rownames = FALSE,
      colnames = c("PROVINCE", "PERIOD", "PNLS", "CORDAID", "COMBINED"),
      options = list(
        dom = 'tp',
        pageLength = 20
      )
    )
  })
  
  output$datatable2 <- DT::renderDataTable({
    dttbl <- df1_all_filt()
    dttbl <- dttbl %>% select(-level_2_name)
    
    DT::datatable(
      dttbl, filter="top", selection="multiple", escape=FALSE,
      rownames = FALSE,
      colnames = c("DISTRICT", "PERIOD", "PNLS", "CORDAID", "COMBINED"),
      options = list(
        dom = 'tp',
        pageLength = 20
      )
    )
  })
  
  
  ## PATIENTS ######################################################################################################
  
  ## DATA 
  
  
  df2_region_filt1 <- reactive({
    m <- df2_region %>%
      filter(
        periods == input$period2)
    m <- left_join(coordinates_region, m, by = c('id' = 'level_2_id'))
    m$value[is.na(m$value)] <- 0
    m$all[is.na(m$all)] <- 0
    m
  })
  
  df2_region_filt2 <- reactive({
    m <- df2_region %>%
      filter(
        periods == input$period2,
        level_2_name == input$region2)
    m$value[is.na(m$value)] <- 0
    m <- m %>% filter(value != 0)
    m
  })
  
  
  ## PLOT 
  
  output$plot4 <- renderPlot({
    
    ggplot(df2_region_filt1())+
      geom_polygon(aes(x  = long, y = lat , 
                       group=id, 
                       fill = all))+
      theme_minimal() +
      coord_map() + 
      scale_fill_gradient(name = "Number of patients",
                          low = '#eeeeee', high = '#ef0404',
                          space = "Lab",
                          na.value = "grey50", guide = "colourbar")+
      ggtitle("TOTAL PATIENTS")+
      theme(plot.title = element_text(size=20, hjust = 0.55),
            legend.position = "bottom",
            legend.key.width = unit(2, "cm"),
            legend.text=element_text(size=13),
            legend.title=element_text(size=15))
    
  })
  
  
  output$plot5 <- renderPlot({
    
    df <- df2_region_filt2()
    
    ggplot(df, aes(x="", y=value, fill=line)) +
      geom_bar(width = 1, stat = "identity") +
      coord_polar("y") +
      blank_theme +
      theme(axis.text.x=element_blank(),
            legend.key.width = unit(2, "cm"),
            legend.text=element_text(size=13),
            legend.title=element_text(size=15)) + 
      scale_fill_manual(name = "Treatment", values=group.colors)
    
    
  })
  
  output$print1 <- renderText({
    paste0("Total number of patients for ", input$period2," (", input$region2, ") :")
  })
  
  output$print2 <- renderText({
    m <- df2_region_filt2()
    n_patients <- m$all[1]
    n_patients
  })
  
  
  
  ## FACILITIES ######################################################################################################
  
  ## DATA
  
  df4_region_filt1 <- reactive({
    m <- df4_region %>%
      filter(
        periods == input$period3)
    m <- left_join(coordinates_region, m, by = c('id' = 'level_2_id'))
    m
  })                  
  
  ## DATA
  
  output$plot6 <- renderPlot({
    
    df <- df3_region
    df <- left_join(coordinates_region, df, by = c('id' = 'level_2_id'))
    
    ggplot(df)+
      geom_polygon(aes(x  = long, y = lat ,
                       group=id,
                       fill = value))+
      theme_minimal() +
      coord_map() +
      scale_fill_gradient(name = "Number of facilities",
                          low = '#eeeeee', high = '#4D71A3',
                          space = "Lab",
                          na.value = "grey50", guide = "colourbar")+
      ggtitle("FACILITIES")+
      theme(plot.title = element_text(size=20, hjust = 0.55),
            legend.position = "bottom",
            legend.key.width = unit(2, "cm"),
            legend.text=element_text(size=13),
            legend.title=element_text(size=15))
    
  })
  
  output$plot7 <- renderPlot({
    
    df <- df4_region_filt1()
    
    ggplot(df)+
      geom_polygon(aes(x  = long, y = lat ,
                       group=id,
                       fill = percentage))+
      theme_minimal() +
      coord_map() +
      scale_fill_gradient(name = "% of facilities",
                          low = '#228b22', high = '#ef0404',
                          space = "Lab",
                          na.value = "grey90", guide = "colourbar",
                          limits = c(0,1))+
      ggtitle("% OF FACILITIES WITH STOCKOUTS")+
      theme(plot.title = element_text(size=20, hjust = 0.55),
            legend.position = "bottom",
            legend.key.width = unit(2, "cm"),
            legend.text=element_text(size=13),
            legend.title=element_text(size=15))
    
  })
  
  plot8_select <- reactive({
    m <- df5_region %>%
      filter(
        stand_name == input$stand_name,
        namelevel2 == input$province_conso)
    m
  })  
  
  output$plot8 <- renderPlot({
    
    df <- plot8_select()
    
    
    
    ggplot(df)+
      geom_line(aes(x  = period, y = value))+
      geom_smooth(aes(x  = period, y = value), method='loess')+
      theme_minimal() +
      ggtitle("Consumption Timeline")+
      theme(plot.title = element_text(size=20, hjust = 0.55),
            legend.position = "bottom",
            legend.key.width = unit(2, "cm"),
            legend.text=element_text(size=13),
            legend.title=element_text(size=15))
    
  })
  
  
  
  plot9_select <- reactive({
    m <- df5_region %>%
      filter(
        stand_name == input$stand_name,
        period == input$period_conso)
    m <- left_join(coordinates_region, m, by = c('id' = 'uidlevel2'))
    m
  })  
  
  output$plot9 <- renderPlot({
    
    df <- plot9_select()
    
    ggplot(df)+
      geom_polygon(aes(x  = long, y = lat ,
                       group=id,
                       fill = value))+
      theme_minimal() +
      coord_map() +
      scale_fill_gradient(name = "Number of dispensed boxes",
                          low = '#eeeeee', high = '#ef0404',
                          space = "Lab",
                          na.value = "grey90", guide = "colourbar")+
      ggtitle("Total number of dispensed boxes")+
      theme(plot.title = element_text(size=20, hjust = 0.55),
            legend.position = "bottom",
            legend.key.width = unit(2, "cm"),
            legend.text=element_text(size=13),
            legend.title=element_text(size=15))
    
  })
    

  
}

In [ ]:
## ui

shinyUI(
  
  mainPanel(
    br(),
    tabsetPanel(
      
      ## PANEL 1
      tabPanel(
        "Source",
        fluidPage(
          br(),
          fluidRow(
            
            column(6,
                   selectInput("period", "Period", 
                               choices = unique(df1$period), 
                               selected = unique(df1$period)[-1])
            )
          ),
          br(),
          fluidRow(
            
            column(4,
                   plotOutput("plot1", height = "600px")),
            column(4,
                   plotOutput("plot2", height = "600px")),
            column(4,
                   plotOutput("plot3", height = "600px"))
          ),
          br(),
          fluidRow(
            DT::dataTableOutput('datatable1', height = "400px")
          ),
          br(),
          fluidRow(
            column(6,
                   selectInput("region", "Region", 
                               choices = unique(df1$level_2_name), 
                               selected = unique(df1$level_2_name)[1])
            )
          ),
          br(),
          fluidRow(
            DT::dataTableOutput('datatable2', height = "400px")
          )
        )
      ),
      
      ## PANEL 2
      tabPanel(
        "Patients",
        fluidPage(
          br(),
          fluidRow(
            
            column(6,
                   selectInput("period2", "Period",
                               choices = unique(df2_region$period),
                               selected = unique(df2_region$period)[-1])
            ),
            column(6,
                   selectInput("region2", "Region",
                               choices = unique(df2_region$level_2_name),
                               selected = unique(df2_region$level_2_name)[1])
            )
          ),
          br(),
          br(),
          fluidRow(
            
            column(6,
                   plotOutput("plot4", height = "600px")),
            column(6,
                   textOutput("print1"),
                   textOutput("print2"),
                   tags$head(tags$style("#print2{color: black;
                                          font-size: 30px;
                                          font-style: bold;
                                          }")),
                   plotOutput("plot5", height = "600px"))
          )
        )
      ),
      
      
      ## PANEL 3
      tabPanel(
        "Facilities",
        fluidPage(
          br(),
          fluidRow(
            column(6,
                   plotOutput("plot6", height = "600px")),
            column(6,
                   selectInput("period3", "Period",
                               choices = unique(df4_region$period),
                               selected = unique(df4_region$period)[-1]),
                   plotOutput("plot7", height = "600px"))
          )
        )
      ),
      
      ## PANEL 4
      tabPanel(
        "Drugs",
        fluidPage(
          br(),
          fluidRow(column(4,selectInput("period_conso", "Period",
                                        choices = sort(unique(df5_region$period)),
                                        selected = sort(unique(df5_region$period))[1])
                          ),
                   column(4, selectInput("stand_name", "Drug",
                               choices = sort(unique(df5_region$stand_name)),
                               selected = 'tdf/3tc/efv(300/300/600 mg) - 30 ces')
                          ),
                   column(4,selectInput("province_conso", "Province",
                               choices = sort(unique(df5_region$namelevel2)),
                               selected = sort(unique(df5_region$namelevel2))[1])
                          )
                   ),
          fluidRow(
            column(6,
                   plotOutput("plot9", height = "600px")),
            column(6,
                   plotOutput("plot8", height = "600px"))
          )
        )
      )
      
      
    ), width = 12))

